In [1]:
import pandas as pd
import numpy as np
import random

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
raw_parc = pd.read_csv("/Users/angelwadhawan/Documents/WCH/datasets/Appendicitis Risk Calculation Data Clean OCT23.csv")
raw_res = pd.read_csv("/Users/angelwadhawan/Documents/WCH/datasets/appendicities.csv")
raw_parc

,ID,URN,DOB,Date Seen,Age (yrs),Gender,Duration of Pain,Transfer,RLQ Tenderness,RLQ Tenderness with Cough/Percussion/Hopping,...,ANC >7.5,CRP Below Detection,Ultrasound Dx: Appendicitis,Pathology Dx: Perforation,Negative Appendectomy,Confirmed Appendicitis,pARC Score (linear predictor),pARC predicted probability,PAS Score,Modified Alvarado Score
0,2,750389,7/10/2013,4/7/2021,8,Male,24-48,No,Yes,No,...,No,0.0,No,No,No,No,-2.0,0.0,4.0,4.0
1,3,806038,26/3/2015,2/7/2021,6,Female,48-96,No,Yes,No,...,No,0.0,No,No,No,No,-4.0,0.0,4.0,4.0
2,4,587987,21/8/2012,1/7/2021,9,Male,24-48,No,Yes,No,...,No,0.0,No,No,No,No,-3.0,0.0,3.0,3.0
3,7,746726,31/10/2009,30/6/2021,12,Male,<24,No,Yes,Yes,...,No,NaN,No,No,No,No,-4.0,0.0,NaN,NaN
4,9,686305,9/11/2009,3/7/2021,12,Female,<24,Yes,Yes,Yes,...,No,0.0,No,No,No,No,-4.0,0.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,714,280024161,11/1/2016,1/8/2022,7,Female,24-48,No,Yes,No,...,No,NaN,NaN,No,No,No,-7.0,0.0,NaN,NaN
681,715,280024317,29/1/2012,2/8/2022,11,Male,24-48,No,Yes,Yes,...,No,0.0,Yes,No,No,Yes,0.0,0.0,6.0,4.0
682,718,787336,21/12/2011,3/8/2022,11,Female,24-48,Yes,Yes,No,...,Yes,0.0,Yes,No,No,Yes,0.0,0.0,7.0,8.0
683,719,565334,16/9/2011,5/8/2022,11,Male,<24,No,Yes,Yes,...,No,0.0,Yes,No,No,Yes,-1.0,0.0,5.0,5.0


In [3]:
filter_raw_parc = raw_parc.loc[:, ["ID", "pARC predicted probability"]]
filter_raw_parc

,ID,pARC predicted probability
0,2,0.0
1,3,0.0
2,4,0.0
3,7,0.0
4,9,0.0
...,...,...
680,714,0.0
681,715,0.0
682,718,0.0
683,719,0.0


In [4]:
filter_res = raw_res.loc[:, ["ID", "Operation Performed"]]
filter_res

,ID,Operation Performed
0,2,No
1,3,No
2,4,No
3,7,No
4,9,No
...,...,...
718,714,NaN
719,715,Yes
720,718,Yes
721,719,Yes


In [5]:
merge_df = pd.merge(filter_raw_parc, filter_res, on="ID")
merge_df = merge_df.dropna(axis=0)
operation_mapping = {'No' : 0, 'Yes': 1}
merge_df['Operation Performed'] = merge_df['Operation Performed'].map(operation_mapping)
merge_df

,ID,pARC predicted probability,Operation Performed
0,2,0.0,0
1,3,0.0,0
2,4,0.0,0
3,7,0.0,0
4,9,0.0,0
...,...,...,...
679,712,1.0,0
681,715,0.0,1
682,718,0.0,1
683,719,0.0,1


In [6]:
train, test = train_test_split(merge_df, test_size=0.2, random_state=42)

In [7]:
print(test.shape, train.shape)

(113, 3) (451, 3)


In [8]:
# function that prints different scores for the model prediction
def get_scores(y_test, y_pred_test):

    # getting the confusion matrix
    tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred_test).ravel()

    # printing out the parameters that we need
    print("Sensitivity is: " + str((tp/(tp+fn))))
    print("Specificity is: " + str((tn/(tn+fp))))
    print("Positive predictive value is: " + str((tp/(tp+fp))))
    print("Accuracy is: " + str((tn/(tn+fn))))

In [9]:
get_scores(test["Operation Performed"], test["pARC predicted probability"])

Sensitivity is: 0.5483870967741935
Specificity is: 0.926829268292683
Positive predictive value is: 0.7391304347826086
Accuracy is: 0.8444444444444444


In [13]:
print(classification_report(test["Operation Performed"], test["pARC predicted probability"]))

              precision    recall  f1-score   support

           0       0.84      0.93      0.88        82
           1       0.74      0.55      0.63        31

    accuracy                           0.82       113
   macro avg       0.79      0.74      0.76       113
weighted avg       0.82      0.82      0.81       113



In [10]:
get_scores(train["Operation Performed"], train["pARC predicted probability"])

Sensitivity is: 0.52
Specificity is: 0.9239130434782609
Positive predictive value is: 0.8125
Accuracy is: 0.7522123893805309


In [14]:
print(classification_report(train["Operation Performed"], train["pARC predicted probability"]))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83       276
           1       0.81      0.52      0.63       175

    accuracy                           0.77       451
   macro avg       0.78      0.72      0.73       451
weighted avg       0.78      0.77      0.75       451



In [11]:
get_scores(merge_df["Operation Performed"], merge_df["pARC predicted probability"])

Sensitivity is: 0.5242718446601942
Specificity is: 0.9245810055865922
Positive predictive value is: 0.8
Accuracy is: 0.7715617715617715


In [15]:
print(classification_report(merge_df["Operation Performed"], merge_df["pARC predicted probability"]))

              precision    recall  f1-score   support

           0       0.77      0.92      0.84       358
           1       0.80      0.52      0.63       206

    accuracy                           0.78       564
   macro avg       0.79      0.72      0.74       564
weighted avg       0.78      0.78      0.77       564

